# An attempt to look at the basic sentiments for facebook comments on a post

This is an experiment, and can actually not be used in real life, as facebooks regulations do not allow for this. So think of it as inspiration, not something that you nor I can actually run and use... unfortunately. 

Anyhow, the reason I wanted to do this was to see how a politicians post, asking people if we should allow immigigants into the country, went. I had a feeling he wouldnt get the support he was aiming for. 


In [ ]:
#Load what's needed
import pandas as pd
import selenium
import time
from selenium import webdriver
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set()


First we need to connect to chrome using the webdriver and selenium.

In [ ]:
# Connect to chrome 
driver = webdriver.Chrome('C:\\Users\\Peter\\AppData\\Local\\Temp\\Rar$EXa19652.30181\\chromedriver.exe')  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/xhtml');
time.sleep(5) # Let the user actually see something!


Next up, we would have to go to the page where we want to extract the comments, and start by clicking to show more comments. The xpath and class names might have to be changed as I am not certain of how the facebook classes etc actually work. 
In order to get all the comments, we need to keep clicking view more comments. Rumour has it that you can 'only' expand to 5.000 comments, so i've added a stop for that.

In [ ]:
#Keep clicking to show more comments while still possible
i = 0
max_comments = 5000
shown = 0
total = 10
print("starting with {} shown and {} total".format(shown, total))
while shown < total :
    if total > 5000:
        total = 5000
    for element in driver.find_elements_by_class_name('_3bu3._293g'):
        print(element.text)
        shown= int(element.text.split(' of ')[0].replace(',', ''))
        total= int(element.text.split(' of ')[1].replace(',', ''))
        print(shown, total)
        # click submit button
        submit_button = driver.find_elements_by_xpath('//*[@id="u_fetchstream_10_13"]/div/div[3]/div[3]/div/a')[0]
        submit_button.click()

print("now with {} shown and {} total".format(shown, total))

Time to grab all the comments and put them in a dataframe, and then save them. Again, the class name might differ...

In [ ]:
df = pd.DataFrame(columns=['Comment', 'html'])
i = 0
for element in driver.find_elements_by_class_name('_6qw3'):
    df = df.append({'Comment': element.text, 'html':element.get_attribute('innerHTML') }, ignore_index=True)
    i+=1
    print(i)
df.head()
df.to_pickle('dataset.pkl') 
df.to_csv('dataset.csv')

And time to have a quick look. Just checking if the text has a 'Ja' (Yes) or 'Nej' (No) in it and then using that to put the comments in buckets so we can draw a graph

In [ ]:
df_raw = pd.read_pickle('dataset.pkl')
df_raw['lower'] = df_raw['Comment'].str.lower()
df_raw['yes'] = df_raw['lower'].str.contains('ja')
df_raw['no'] = df_raw['lower'].str.contains('nej')
df_raw['answer'] = np.where(df_raw['no'] == True, 0, 2)
df_raw['answer'] = np.where(df_raw['yes'] == True, 1, df_raw['answer'])
df_raw.sample(20)

And then it's time to draw a quick graph....

In [ ]:
bins = (np.arange(0,4,1))
plt.hist(df_raw['answer'], bins=bins);
plt.xticks((0.5,1.5,2.5),('No', 'Yes','Not Clear'));
plt.savefig('output.png')